<a href="https://colab.research.google.com/github/PhilippBern/datascience/blob/master/data_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pathlib import Path
from datetime import datetime
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from collections import deque

In [ ]:
def extract_time_info(name: str) -> tuple[datetime, int]:
    time_information = name[0:-5].split("_")
    date = int(time_information[0])
    date = datetime.fromtimestamp(date)
    nanosec = int(time_information[1])
    return (date, nanosec)

In [ ]:
def extract_dict_x_y_z(dict: dict) -> tuple[float, float, float]:
    return (dict["x"], dict["y"], dict["z"])

In [ ]:
def build_dataframe(df: pd.DataFrame, data_name: str) -> pd.DataFrame:
    # initial empty list
    data_list = list()

    for i in range(df.shape[0]):
        current_entry = df.iloc[i]

        object_id = current_entry.obj_id
        object_type = current_entry.obj_type

        psr = current_entry.psr

        pos_dict = psr["position"]
        pos_x, pos_y, pos_z = extract_dict_x_y_z(dict=pos_dict)

        scale_dict = psr["scale"]
        scale_x, scale_y, scale_z = extract_dict_x_y_z(dict=scale_dict)

        rotation_dict = psr["rotation"]
        rotation_x, rotation_y, rotation_z = extract_dict_x_y_z(dict=rotation_dict)

        unix_timestamp, nanosec = extract_time_info(name=data_name)

        new_entry = {
                    "object_id": object_id,
                    "object_type": object_type,
                    "unix_timestamp": unix_timestamp,
                    "nanosec": nanosec,
                    "pos_x": pos_x,
                    "pos_y": pos_y,
                    "pos_z": pos_z,
                    "scale_x": scale_x,
                    "scale_y": scale_y,
                    "scale_z": scale_z,
                    "rotation_x": rotation_x,
                    "rotation_y": rotation_y,
                    "rotation_z": rotation_z
                }

        data_list.append(new_entry)

    new_dataframe = pd.DataFrame(data_list)

    return new_dataframe

In [ ]:
# Work directory
cwd = Path.cwd()

data_path = cwd / "data" / "Regensburg" / "bus_station" / "1730708754_289317488"
print(data_path)
df_list = list()

name = None

for data in data_path.iterdir():
        if data.is_file():
            name = data.name
            df = pd.read_json(data)
            transformed_df = build_dataframe(df=df, data_name=name)
            df_list.append(transformed_df)

/content/data/Regensburg/bus_station/1730708754_289317488


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/Regensburg/bus_station/1730708754_289317488'

In [ ]:
big_df = pd.concat(df_list, ignore_index=True, axis=0)


In [ ]:
big_df.to_csv("bus_station.csv", index=False)

In [ ]:
test_df = pd.read_csv("bus_station.csv")

test_df

In [ ]:
INPUT_CSV = "bus_station.csv"
OUT_GIF   = "lidar.gif"
FPS       = 5
TRAIL_LEN = 5
LABEL_OFFSET = (0.2, 0.2)

df = pd.read_csv(INPUT_CSV)

dt = pd.to_datetime(df["unix_timestamp"], errors="coerce")
ns = pd.to_timedelta(df["nanosec"], unit="ns")
df["t"] = dt + ns
df = df.dropna(subset=["t"]).copy()
df = df.sort_values("t").reset_index(drop=True)

df = df.rename(columns={"pos_x": "x", "pos_y": "y"})

times = df["t"].sort_values().unique()

xmin, xmax = df["x"].min(), df["x"].max()
ymin, ymax = df["y"].min(), df["y"].max()
dx = (xmax - xmin) if xmax > xmin else 1.0
dy = (ymax - ymin) if ymax > ymin else 1.0
pad_x, pad_y = 0.05 * dx, 0.05 * dy

object_ids = sorted(df["object_id"].unique())
trails = {oid: deque(maxlen=TRAIL_LEN) for oid in object_ids}

latest_type = {oid: None for oid in object_ids}

grouped_by_time = dict(tuple(df.groupby("t")))

#Plot Setup
fig, ax = plt.subplots(figsize=(6, 6))
ax.set_title("Objekt Bewegungen")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_aspect("equal", adjustable="box")
ax.set_xlim(xmin - pad_x, xmax + pad_x)
ax.set_ylim(ymin - pad_y, ymax + pad_y)

point_artists = {}
trail_artists = {}
label_artists = {}

for oid in object_ids:
    point, = ax.plot([], [], marker="o", linestyle="", label=str(oid))
    trail, = ax.plot([], [], linewidth=1.0, alpha=0.7)
    txt = ax.text(0, 0, "", fontsize=8, ha="left", va="bottom")
    point_artists[oid] = point
    trail_artists[oid] = trail
    label_artists[oid] = txt

#ax.legend(title="object_id", loc="upper right", fontsize="small", ncol=1, frameon=True)
time_text = ax.text(0.02, 0.97, "", transform=ax.transAxes, va="top")

def init():
    for oid in object_ids:
        point_artists[oid].set_data([], [])
        trail_artists[oid].set_data([], [])
        label_artists[oid].set_text("")
        label_artists[oid].set_position((0, 0))
        trails[oid].clear()
    time_text.set_text("")
    return list(point_artists.values()) + list(trail_artists.values()) + list(label_artists.values()) + [time_text]

def update(frame_idx):
    current_time = times[frame_idx]
    frame_df = grouped_by_time.get(current_time, pd.DataFrame(columns=df.columns))
    present_ids = set(frame_df["object_id"].unique())
    xoff, yoff = LABEL_OFFSET

    for oid in object_ids:
        if oid in present_ids:
            row = frame_df[frame_df["object_id"] == oid].iloc[-1]
            x, y = row["x"], row["y"]
            trails[oid].append((x, y))

            point_artists[oid].set_data([x], [y])

            if len(trails[oid]) >= 2:
                xs, ys = zip(*trails[oid])
                trail_artists[oid].set_data(xs, ys)
            else:
                trail_artists[oid].set_data([], [])

            latest_type[oid] = row.get("object_type", latest_type.get(oid))
            if latest_type[oid] is None or pd.isna(latest_type[oid]):
                label_text = f"{oid}"
            else:
                label_text = f"{oid} ({latest_type[oid]})"
            label_artists[oid].set_text(label_text)
            label_artists[oid].set_position((x + xoff, y + yoff))
        else:
            point_artists[oid].set_data([], [])
            label_artists[oid].set_text("")
            if len(trails[oid]) >= 2:
                xs, ys = zip(*trails[oid])
                trail_artists[oid].set_data(xs, ys)
            else:
                trail_artists[oid].set_data([], [])


    time_text.set_text(f"t = {pd.to_datetime(current_time).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}")
    return list(point_artists.values()) + list(trail_artists.values()) + list(label_artists.values()) + [time_text]

interval_ms = 1000 / FPS
anim = animation.FuncAnimation(
    fig,
    update,
    init_func=init,
    frames=len(times),
    interval=interval_ms,
    blit=True
)

gif_ok = True
try:
    gif_writer = animation.PillowWriter(fps=FPS)
    anim.save(OUT_GIF, writer=gif_writer)
except Exception as e:
    gif_ok = False